# 🤖 AlphaTraderLab PPO v1 - Training & Evaluation

---

## Welcome to Step 2!

In **Step 1**, we built a trading environment and tested it with a random agent.

In **Step 2** (this notebook), we'll:
1. **Train** a smart RL agent using PPO (Proximal Policy Optimization)
2. **Evaluate** it against baselines (random agent, buy & hold)
3. **Visualize** and compare the results

### 🧠 What is PPO?

**PPO (Proximal Policy Optimization)** is a state-of-the-art reinforcement learning algorithm that:
- Learns by trying different actions and seeing what works
- Uses a neural network to map observations (market data) → actions (FLAT/LONG/SHORT)
- Updates the network gradually to improve trading decisions
- Is stable, reliable, and widely used in RL research

Think of it as teaching a robot trader by letting it practice thousands of times, learning from its mistakes.

### 📊 What We'll Do

1. **Download** Bitcoin price data (2018-present)
2. **Split** into training (70%) and testing (30%) periods
3. **Train** PPO agent on training data
4. **Evaluate** on test data:
   - PPO agent (our trained model)
   - Random agent (baseline)
   - Buy & Hold (benchmark)
5. **Compare** performance with charts and tables

Let's get started! 🚀

---

## 📦 Step 1: Setup and Installation

First, we'll install all required packages and import libraries.

In [ ]:
# Detect if we're running in Google Colab
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Running in Google Colab - Installing dependencies...")
    
    # Install required packages
    !pip install -q numpy pandas matplotlib yfinance gymnasium stable-baselines3
    
    print("✅ Installation complete!")
else:
    print("💻 Running locally - Make sure you've installed requirements.txt")
    print("   Run: pip install -r requirements.txt")

## 📚 Step 2: Import Libraries

Import all necessary Python libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import warnings
import os

# Stable-Baselines3 imports
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All libraries imported successfully!")

## 🔧 Step 3: Load the Trading Environment

We'll load our custom `TradingEnv` from Step 1.

**Note**: In Colab, you'll need to upload `trading_env.py` and `evaluation.py`.

In [ ]:
if IN_COLAB:
    print("📁 In Colab: Please upload the required files")
    print("   1. trading_env.py (from envs/ folder)")
    print("   2. evaluation.py (from utils/ folder)")
    print()
    
    from google.colab import files
    
    # Upload trading_env.py
    print("👉 Upload 'trading_env.py' first:")
    uploaded = files.upload()
    
    # Upload evaluation.py
    print("\n👉 Upload 'evaluation.py' next:")
    uploaded = files.upload()
    
    print("\n✅ Files uploaded!")
    
    # Import from uploaded files
    from trading_env import TradingEnv
    from evaluation import (
        evaluate_agent,
        evaluate_random_agent,
        evaluate_buy_and_hold,
        compare_agents
    )
    
else:
    print("💻 Running locally - Importing from project structure")
    
    # Add parent directory to path
    import sys
    import os
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    
    # Import from project structure
    from envs.trading_env import TradingEnv
    from utils.evaluation import (
        evaluate_agent,
        evaluate_random_agent,
        evaluate_buy_and_hold,
        compare_agents
    )

print("✅ TradingEnv and evaluation utilities loaded!")

## 📊 Step 4: Download and Split Data

We'll download Bitcoin historical data and split it into:
- **Training set** (70%): Used to train the PPO agent
- **Testing set** (30%): Used to evaluate all agents

### 🤔 Why Split?

We want to see if our agent can generalize to **unseen** data. Training and testing on the same data would be cheating - the agent would just memorize the answers!

Think of it like studying for an exam: you practice on some problems (training), then take a test with new problems (testing).

In [ ]:
print("📡 Downloading BTC-USD historical data...")
print("   This may take 10-20 seconds...")
print()

# Download Bitcoin data
ticker = "BTC-USD"
start_date = "2018-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

df = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)

print(f"✅ Downloaded {len(df)} days of {ticker} data")
print(f"   Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")

# Split into train (70%) and test (30%)
train_ratio = 0.7
split_index = int(len(df) * train_ratio)

train_df = df.iloc[:split_index].copy()
test_df = df.iloc[split_index:].copy()

print(f"\n📈 Data Split:")
print(f"   Training:   {len(train_df):4d} days ({train_df.index[0].strftime('%Y-%m-%d')} to {train_df.index[-1].strftime('%Y-%m-%d')})")
print(f"   Testing:    {len(test_df):4d} days ({test_df.index[0].strftime('%Y-%m-%d')} to {test_df.index[-1].strftime('%Y-%m-%d')})")

# Show first few rows of training data
print("\n📋 Training data preview:")
print(train_df.head())

## 📈 Step 5: Visualize the Train/Test Split

Let's see what the data split looks like on a chart.

In [ ]:
plt.figure(figsize=(14, 6))

# Plot training data
plt.plot(train_df.index, train_df['Close'], label='Training Data', color='#3498db', linewidth=2)

# Plot testing data
plt.plot(test_df.index, test_df['Close'], label='Testing Data', color='#e74c3c', linewidth=2)

# Add vertical line at split
plt.axvline(x=train_df.index[-1], color='gray', linestyle='--', linewidth=2, label='Train/Test Split')

plt.title('📊 BTC-USD Price: Train/Test Split', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 The agent will train on the blue section and be tested on the red section.")

## 🎮 Step 6: Create Training and Testing Environments

We'll create two environments:
1. **Training environment**: Uses `train_df`
2. **Testing environment**: Uses `test_df`

### 🔧 VecEnv Wrapper

Stable-Baselines3 requires environments to be "vectorized" (wrapped in `DummyVecEnv`). This allows for efficient batched computation, even though we're only using one environment.

Think of it as putting your environment in a special container that SB3 knows how to work with.

In [ ]:
# Configuration
WINDOW_SIZE = 30
INITIAL_BALANCE = 10000.0
TRANSACTION_COST = 0.001  # 0.1%

print("🎮 Creating environments...")
print(f"   Window size: {WINDOW_SIZE} days")
print(f"   Initial balance: ${INITIAL_BALANCE:,.2f}")
print(f"   Transaction cost: {TRANSACTION_COST*100:.2f}%")
print()

# Helper function to create a vectorized environment
def make_env(df, window_size=WINDOW_SIZE, initial_balance=INITIAL_BALANCE):
    """
    Create a TradingEnv and wrap it in DummyVecEnv for Stable-Baselines3.
    
    This wrapper is required by SB3 for all environments.
    """
    def _init():
        return TradingEnv(
            df=df,
            window_size=window_size,
            initial_balance=initial_balance,
            transaction_cost=TRANSACTION_COST
        )
    return DummyVecEnv([_init])

# Create training environment (vectorized)
train_env = make_env(train_df)
print("✅ Training environment created (vectorized for SB3)")

# Create testing environment (non-vectorized for easier evaluation)
test_env = TradingEnv(
    df=test_df,
    window_size=WINDOW_SIZE,
    initial_balance=INITIAL_BALANCE,
    transaction_cost=TRANSACTION_COST
)
print("✅ Testing environment created")

# Show environment info
print(f"\n📊 Environment details:")
print(f"   Observation space: {test_env.observation_space.shape}")
print(f"   Action space: {test_env.action_space} (0=FLAT, 1=LONG, 2=SHORT)")

## 🤖 Step 7: Train the PPO Agent

Now comes the exciting part - training our RL agent!

### 🧠 How PPO Learns

1. **Observe**: Look at market data (30 days of OHLCV)
2. **Act**: Choose an action (FLAT, LONG, or SHORT)
3. **Reward**: Get feedback (did we make or lose money?)
4. **Learn**: Update the neural network to make better decisions
5. **Repeat**: Do this thousands of times

### ⏱️ Training Time

- **100,000 steps**: ~3-5 minutes on Colab (GPU)
- **200,000 steps**: ~6-10 minutes (better results)
- **50,000 steps**: ~2-3 minutes (quick test)

We'll start with 100,000 steps as a good balance between time and performance.

### 📚 Hyperparameters Explained

- **learning_rate** (3e-4): How fast the agent learns (too high = unstable, too low = slow)
- **n_steps** (2048): Steps before each update
- **batch_size** (64): Size of minibatches for training
- **gamma** (0.99): How much to value future rewards
- **ent_coef** (0.01): Encourages exploration

These are standard values that work well for most problems. You can experiment later!

In [ ]:
# Training configuration
TOTAL_TIMESTEPS = 100_000  # Reduce to 50_000 for faster testing

print("🤖 Creating PPO agent...")
print()

# Create PPO model
model = PPO(
    policy="MlpPolicy",           # Multi-Layer Perceptron (neural network)
    env=train_env,                # Training environment
    learning_rate=3e-4,           # Learning rate
    n_steps=2048,                 # Steps before update
    batch_size=64,                # Minibatch size
    n_epochs=10,                  # Training epochs per update
    gamma=0.99,                   # Discount factor
    gae_lambda=0.95,              # GAE parameter
    clip_range=0.2,               # PPO clipping parameter
    ent_coef=0.01,                # Entropy coefficient (exploration)
    verbose=1                      # Print progress
)

print("✅ PPO agent created with standard hyperparameters")
print()
print("🏋️ Training starting...")
print(f"   Total timesteps: {TOTAL_TIMESTEPS:,}")
print(f"   Estimated time: ~{TOTAL_TIMESTEPS // 30000} minutes")
print()
print("📊 Training progress:")
print("-" * 60)

# Train the model
model.learn(total_timesteps=TOTAL_TIMESTEPS)

print("-" * 60)
print("\n✅ Training complete!")

# Save the model
model_path = "ppo_btc_trader.zip"
model.save(model_path)
print(f"💾 Model saved to: {model_path}")

## 📊 Step 8: Evaluate All Agents on Test Data

Now let's see how well our PPO agent performs compared to:
1. **Random Agent**: Chooses actions randomly (worst case baseline)
2. **Buy & Hold**: Buys at the start and holds (common benchmark)

### 🎯 What to Expect

**Good results**:
- PPO > Random (agent learned something!)
- PPO ≈ Buy & Hold or better (competitive performance)

**Bad results**:
- PPO ≈ Random (agent didn't learn)
- PPO << Buy & Hold (agent is worse than doing nothing)

Remember: Trading is HARD! Even professional traders struggle to beat buy & hold consistently.

In [ ]:
print("📊 Evaluating all agents on test data...")
print("="*60)

# Evaluate PPO Agent
print("\n🤖 1. Evaluating PPO Agent (Trained)")
print("-" * 60)
test_env.reset()
ppo_results = evaluate_agent(test_env, model=model, n_episodes=1, deterministic=True)
print(f"   Initial equity: ${ppo_results['initial_equity']:,.2f}")
print(f"   Final equity:   ${ppo_results['final_equity']:,.2f}")
print(f"   Total return:   {ppo_results['total_return']:.2f}%")
print(f"   Total reward:   {ppo_results['total_reward']:.4f}")
print(f"   Trades made:    {ppo_results['n_trades']}")

# Evaluate Random Agent
print("\n🎲 2. Evaluating Random Agent (Baseline)")
print("-" * 60)
test_env.reset()
random_results = evaluate_random_agent(test_env, n_episodes=1)
print(f"   Initial equity: ${random_results['initial_equity']:,.2f}")
print(f"   Final equity:   ${random_results['final_equity']:,.2f}")
print(f"   Total return:   {random_results['total_return']:.2f}%")
print(f"   Total reward:   {random_results['total_reward']:.4f}")
print(f"   Trades made:    {random_results['n_trades']}")

# Evaluate Buy & Hold
print("\n📈 3. Evaluating Buy & Hold (Benchmark)")
print("-" * 60)
test_env.reset()
bh_results = evaluate_buy_and_hold(test_env, n_episodes=1)
print(f"   Initial equity: ${bh_results['initial_equity']:,.2f}")
print(f"   Final equity:   ${bh_results['final_equity']:,.2f}")
print(f"   Total return:   {bh_results['total_return']:.2f}%")
print(f"   Total reward:   {bh_results['total_reward']:.4f}")
print(f"   Trades made:    {bh_results['n_trades']}")

print("\n" + "="*60)
print("✅ Evaluation complete!")

## 📊 Step 9: Comparison Table

Let's create a nice summary table comparing all three agents.

In [ ]:
# Create comparison dictionary
results_dict = {
    'PPO (Trained)': ppo_results,
    'Random (Baseline)': random_results,
    'Buy & Hold (Benchmark)': bh_results
}

# Generate comparison table
comparison_df = compare_agents(results_dict)

print("\n📊 Agent Comparison Table")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Determine winner
best_agent = comparison_df.iloc[0]['Agent']
print(f"\n🏆 Best Performer: {best_agent}")

## 📈 Step 10: Visualize Equity Curves

The most important chart: how does portfolio value change over time for each agent?

### 📊 How to Read This Chart

- **Y-axis**: Portfolio value (equity) in dollars
- **X-axis**: Trading steps (each step = 1 day)
- **Higher line = better performance**
- **Smooth line = consistent strategy**
- **Volatile line = risky strategy**

In [ ]:
# Create figure
plt.figure(figsize=(14, 7))

# Plot PPO agent
plt.plot(ppo_results['history']['step'], 
         ppo_results['history']['equity'],
         label='PPO (Trained)', 
         color='#2ecc71',  # Green
         linewidth=2.5,
         alpha=0.9)

# Plot Random agent
plt.plot(random_results['history']['step'], 
         random_results['history']['equity'],
         label='Random (Baseline)', 
         color='#e74c3c',  # Red
         linewidth=2,
         alpha=0.7,
         linestyle='--')

# Plot Buy & Hold
plt.plot(bh_results['history']['step'], 
         bh_results['history']['equity'],
         label='Buy & Hold (Benchmark)', 
         color='#3498db',  # Blue
         linewidth=2,
         alpha=0.7,
         linestyle='-.')

# Add initial balance line
plt.axhline(y=INITIAL_BALANCE, 
            color='gray', 
            linestyle=':', 
            linewidth=1.5,
            label=f'Initial Balance (${INITIAL_BALANCE:,.0f})',
            alpha=0.5)

# Customize plot
plt.title('💰 Portfolio Equity Over Time (Test Period)', fontsize=16, fontweight='bold')
plt.xlabel('Step (Days)', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Show plot
plt.show()

print("\n💡 Interpretation:")
print("   - Green line (PPO): Our trained agent")
print("   - Red dashed line (Random): Worst-case baseline")
print("   - Blue dash-dot line (Buy & Hold): Common benchmark")
print("   - Gray dotted line: Starting point ($10,000)")

## 📊 Step 11: Action Distribution Analysis

Let's see what actions each agent took during the test period.

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

action_names = ['FLAT', 'LONG', 'SHORT']
action_colors = ['#95a5a6', '#3498db', '#e74c3c']  # Gray, Blue, Red

# Plot PPO actions
ppo_actions = ppo_results['history']['action']
axes[0].plot(ppo_actions, linewidth=1, color='#2ecc71', alpha=0.7)
axes[0].set_title('🤖 PPO Agent Actions', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Action')
axes[0].set_yticks([0, 1, 2])
axes[0].set_yticklabels(action_names)
axes[0].grid(True, alpha=0.3)

# Plot Random actions
random_actions = random_results['history']['action']
axes[1].plot(random_actions, linewidth=1, color='#e74c3c', alpha=0.7)
axes[1].set_title('🎲 Random Agent Actions', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Action')
axes[1].set_yticks([0, 1, 2])
axes[1].set_yticklabels(action_names)
axes[1].grid(True, alpha=0.3)

# Plot Buy & Hold actions
bh_actions = bh_results['history']['action']
axes[2].plot(bh_actions, linewidth=1, color='#3498db', alpha=0.7)
axes[2].set_title('📈 Buy & Hold Actions', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Action')
axes[2].set_xlabel('Step (Days)')
axes[2].set_yticks([0, 1, 2])
axes[2].set_yticklabels(action_names)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print action statistics
print("\n📊 Action Distribution:")
print("="*60)

for agent_name, results in results_dict.items():
    actions = results['history']['action']
    action_counts = pd.Series(actions).value_counts().sort_index()
    
    print(f"\n{agent_name}:")
    for action_idx in range(3):
        count = action_counts.get(action_idx, 0)
        pct = (count / len(actions)) * 100
        print(f"   {action_names[action_idx]:6s}: {count:4d} times ({pct:5.1f}%)")

## 🎯 Step 12: Performance Summary & Insights

Let's summarize what we learned and interpret the results.

In [ ]:
print("\n" + "="*80)
print("📊 PERFORMANCE SUMMARY")
print("="*80)

# Calculate rankings
returns = {
    'PPO': ppo_results['total_return'],
    'Random': random_results['total_return'],
    'Buy & Hold': bh_results['total_return']
}

sorted_agents = sorted(returns.items(), key=lambda x: x[1], reverse=True)

print("\n🏆 Rankings (by Total Return):")
for i, (agent, ret) in enumerate(sorted_agents, 1):
    emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
    print(f"   {emoji} {i}. {agent:25s}: {ret:7.2f}%")

print("\n" + "-"*80)
print("🎓 WHAT DOES THIS MEAN?")
print("-"*80)

ppo_return = ppo_results['total_return']
random_return = random_results['total_return']
bh_return = bh_results['total_return']

# Interpretation
print("\n📈 PPO vs Random:")
if ppo_return > random_return + 5:
    print("   ✅ GREAT! PPO significantly outperformed random trading.")
    print("   ✅ The agent learned meaningful trading strategies.")
elif ppo_return > random_return:
    print("   ✅ GOOD! PPO beat random, but margin is small.")
    print("   💡 Try training longer or tuning hyperparameters.")
else:
    print("   ⚠️  WARNING! PPO didn't beat random.")
    print("   💡 The agent may need more training or better hyperparameters.")

print("\n📊 PPO vs Buy & Hold:")
if ppo_return > bh_return + 5:
    print("   🎉 EXCELLENT! PPO beat the buy & hold benchmark!")
    print("   🎉 This is hard to achieve - great result!")
elif ppo_return > bh_return:
    print("   ✅ GOOD! PPO slightly beat buy & hold.")
    print("   💡 Results may vary - trading is inherently noisy.")
else:
    print("   ℹ️  PPO didn't beat buy & hold (this is common).")
    print("   ℹ️  Buy & hold is a strong benchmark for trending markets.")

print("\n" + "-"*80)
print("💡 IMPORTANT NOTES")
print("-"*80)
print("1. These results are on a SINGLE test period - not statistically robust.")
print("2. Past performance does NOT guarantee future results.")
print("3. Real trading has additional challenges (slippage, execution, psychology).")
print("4. This is an EDUCATIONAL PROJECT - not investment advice!")

print("\n" + "="*80)
print("✅ STEP 2 COMPLETE!")
print("="*80)

## 🔍 Step 13: Next Steps & Improvements

Congratulations on completing Step 2! Here are ideas for further exploration:

### 🚀 Immediate Improvements
1. **Train longer**: Increase `TOTAL_TIMESTEPS` to 200,000 or 500,000
2. **Multiple runs**: Train several times and average results
3. **Different splits**: Try 80/20 or 60/40 train/test split

### 🧪 Experiments
1. **Other assets**: Try ETH-USD, stocks (AAPL, TSLA), etc.
2. **Window sizes**: Try 10, 20, 50, or 100 day windows
3. **Hyperparameters**: Adjust learning rate, entropy coefficient, etc.
4. **Other algorithms**: Try A2C, DQN, or SAC from Stable-Baselines3

### 🎨 Advanced Features (Step 3?)
1. **Technical indicators**: Add RSI, MACD, Bollinger Bands to observations
2. **Risk metrics**: Calculate Sharpe ratio, max drawdown, etc.
3. **Multiple timeframes**: Combine daily + hourly data
4. **Portfolio optimization**: Trade multiple assets simultaneously
5. **Live paper trading**: Test on real-time data (without real money)

### 📚 Learning Resources
- [Stable-Baselines3 Docs](https://stable-baselines3.readthedocs.io/)
- [PPO Paper](https://arxiv.org/abs/1707.06347) (for the curious)
- [Spinning Up in Deep RL](https://spinningup.openai.com/) (comprehensive RL guide)

---

## 🎉 Congratulations!

You've successfully:
- ✅ Trained a PPO agent from scratch
- ✅ Evaluated it against baselines
- ✅ Visualized and interpreted the results
- ✅ Learned about RL in a practical context

**Remember**: This is for LEARNING, not for making investment decisions!

Keep experimenting, keep learning, and have fun! 🚀📈🤖

---

## 💾 Optional: Save Results

Save your results to CSV files for later analysis.

In [ ]:
# Save equity histories
ppo_results['history'].to_csv('ppo_results.csv', index=False)
random_results['history'].to_csv('random_results.csv', index=False)
bh_results['history'].to_csv('bh_results.csv', index=False)

# Save comparison table
comparison_df.to_csv('agent_comparison.csv', index=False)

print("💾 Results saved:")
print("   - ppo_results.csv")
print("   - random_results.csv")
print("   - bh_results.csv")
print("   - agent_comparison.csv")

# Download files in Colab
if IN_COLAB:
    from google.colab import files
    files.download('ppo_results.csv')
    files.download('agent_comparison.csv')
    files.download(model_path)
    print("\n📥 Files downloaded to your computer!")